In [1]:
import os 
import numpy as np 
import matplotlib.pyplot as plot 
import csv
import scipy.optimize as opt
import time

In [2]:
def get_file_name():
    # Write in the data file location. 
    folder_directory = 'C:\\Users\\Ryan\\Desktop\\20200623_Thermometry'
    date = '20200623'
    file_name = '20200623_Thermometry_analysis'
    return(folder_directory, date, file_name)

In [3]:
def import_data(folder_directory, file_name):
    # read in the data from the date_file. 
    data = []
    with open(folder_directory + '\\' + file_name +'.csv', newline ='') as csvfile:
        file_data = csv.reader(csvfile, delimiter = ',')
        for row in file_data:
            try:
                float_row = [float(x) for x in row]
                data.append(float_row)
            except:
                pass
             
    data = np.array(data)    
    return(data)

In [4]:
def get_ND_numbers(data_set):
    # get the row indicies for the different ND clusters.
    start_index = [0]
    stop_index = []
    for i in range(len(data_set[:,0])):
        if data_set[i,0] != data_set[max(start_index),0]:
            stop_index.append(i-1)
            start_index.append(i)
    
    stop_index.append(len(data_set[:,0]))            
    return(start_index, stop_index)

In [25]:
def scatter_plot(data_set, start_index, stop_index, ND_Cluter):
    plot.rcParams.update({'font.size' : 16})
    plot.rcParams.update({'figure.figsize' :[12, 8]})
    fig1, ax1 = plot.subplots()
    
    temp = data_set[start_index:stop_index,2]
    temp_error = data_set[start_index:stop_index,3]
    res_freq = data_set[start_index:stop_index,4]
    res_error = data_set[start_index:stop_index,5]
    
    
    ax1.errorbar(temp,res_freq, xerr =temp_error, yerr=res_error,
                 fmt='x', label='ND ' + str(ND_Cluster) + ' Data set', color = 'k', linewidth = 2)
    ax1.set_ylabel('Microwave Resonant \n Frequency (GHz)')
    ax1.set_xlabel('Temperature (oC)')
    ax1.legend(loc = 1)
    return(fig1, ax1, temp, res_freq)

In [21]:
def linear_fitting_plot(x,y, ax1):
    ax1.plot(x,y, label='Straight line fit', color='r', linewidth = '2')
    ax1.legend()
    return(ax1)

In [7]:
def linear_fit(x, m, c):
    y = m*x + c
    return(y)

In [8]:
def straight_line_fit(temp, res_freq):
    
    init_params = [1, 3.0]
    opt_cf_params, opt_confidence = opt.curve_fit(linear_fit, 
                                                 temp, res_freq, init_params)
    cf_errors = np.sqrt(np.diag(opt_confidence))
    y = linear_fit(temp, opt_cf_params[0], opt_cf_params[1])
    
    return(opt_cf_params, cf_errors, y)
    

In [18]:
def save_data(ND_cluster, date, opt_cf_params, cf_errors, fig1):
    
    destination_folder = 'C:\\Users\\Ryan\\Desktop\\20200623_Thermometry\\res_freq(T)\\'  
    output_data_file = destination_folder + date + '_res_freq(T).dat'
    # Checking to see if the folder exists in which to save the data, and creating it if it does not
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        # Checking the output file exists and creating and initialising it if not
    if os.path.isfile(output_data_file) == False:
        with open(output_data_file, "w") as wf:
            wf.write('ND Cluster' +  ', \t' + 'dD(T)/dT (kHz/K)' +  ', \t' 
                     + 'error dD(T)/dT (kHz/K)' + ', \t' + 'y intercept (GHz)' + ', \t'
                     + 'error y intercept (GHz)' + '\n')
    # Writing contrast dip description to the .dat file
    
    with open(output_data_file, "a") as wf:        
        wf.write(str(ND_cluster) + ', \t' + str(opt_cf_params[0]*10**6) + ', \t'
                 + str(cf_errors[0]*10**6) + ', \t' + 
                 str(opt_cf_params[1]) + ', \t' + str(cf_errors[1]))
        wf.write('\n')

#    # Saving a plot to a folder. 
    fig1.savefig(destination_folder + date + '_' + 'ND_Cluster_' + 
                 str(ND_cluster) + '_Temp_gradient'+'.pdf', transparent = True)

In [27]:
folder_name, date, file_name = get_file_name()
data_set = import_data(folder_name, file_name)
start_index, stop_index = get_ND_numbers(data_set)

for i in range(len(start_index)):
    ND_Cluster = int(data_set[start_index[i],0])
    fig1, ax1,temp, res_freq = scatter_plot(data_set,start_index[i],stop_index[i]+1, ND_Cluster)
    opt_cf_params, cf_errors, fitted_y = straight_line_fit(temp, res_freq)
    linear_fitting_plot(temp, fitted_y, ax1)
    save_data(ND_Cluster, date, opt_cf_params, cf_errors, fig1)
    time.sleep(0.1)
    plot.close(fig1)
